<a href="https://colab.research.google.com/github/Spandana2704/ML/blob/main/ML(W4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np

In [27]:
df=pd.read_csv('/content/titanic_train.csv')
df.shape

(891, 12)

In [28]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [29]:
df=df.drop(['PassengerId','Name','Ticket'],axis=1)

In [30]:
df.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [31]:
[col for col in df.columns if df[col].isnull().sum()>0]

['Age', 'Cabin', 'Embarked']

In [32]:
df['Age'] = df['Age'].fillna(df['Age'].median())

In [33]:
df=df.drop(['Embarked','Cabin'],axis=1)

In [34]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [35]:
from sklearn.metrics import roc_auc_score

In [36]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [37]:
X = df.drop('Survived',axis=1)
y = df['Survived']

In [38]:
X = pd.get_dummies(X,drop_first=True)

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [42]:
grouped_feature_ls = []
correlated_groups = []
def correlation (dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname=corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr
corr_features=correlation (x_test, 0.9)
correlated_features = set(corr_features)
print(len(correlated_features))

0


In [43]:
sfs1=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=True,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)

In [44]:
sfs1=sfs1.fit(np.array(x_train.fillna(0)),y_train)

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    3.9s finished

[2025-08-19 15:17:35] Features: 1/5 -- score: 0.765194136612553[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.3s finished

[2025-08-19 15:17:39] Features: 2/5 -- score: 0.832223720877483[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.6s finished

[2025-08-19 15:17:41] Features: 3/5 -- score: 0.8391936829655554[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.5s finished

[2025-08-19 15:17:44] Features: 4/5 -- score: 0.8452480476977415[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.4s finished

[2025-08-19 15:17:45] Features: 5/5 -- score: 0.8450046174783709

In [45]:
print('best accracy score',sfs1.k_score_)
print('best subset(indices)',sfs1.k_feature_idx_)
print('Best subset (corresponding names):', sfs1.k_feature_names_)

best accracy score 0.8450046174783709
best subset(indices) (0, 1, 3, 4, 5)
Best subset (corresponding names): ('0', '1', '3', '4', '5')


In [46]:
sfs2=SFS(RandomForestClassifier(n_jobs=4),k_features=5,forward=False,verbose=2,scoring='roc_auc' or 'accuracy',cv=3)
sfs2=sfs2.fit(np.array(x_train.fillna(0)),y_train)
print('best accracy score',sfs2.k_score_)
print('best subset(indices)',sfs2.k_feature_idx_)
print('Best subset (corresponding names):', sfs2.k_feature_names_)

best accracy score 0.8440774764265578
best subset(indices) (0, 1, 2, 4, 5)
Best subset (corresponding names): ('0', '1', '2', '4', '5')


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    4.1s finished

[2025-08-19 15:18:28] Features: 5/5 -- score: 0.8440774764265578

In [47]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
efs=EFS(RandomForestClassifier(n_jobs=4),min_features=1,max_features=4,scoring='accuracy',cv=5)
efs=efs.fit(x_train,y_train)
print('best accuracy score',efs.best_score_)
print('best subset(indices)',efs.best_idx_)
print('Best subset (corresponding names):', efs.best_feature_names_)

Features: 56/56

best accuracy score 0.8089806451612903
best subset(indices) (0, 4, 5)
Best subset (corresponding names): ('Pclass', 'Fare', 'Sex_male')


In [48]:
efs_results = pd.DataFrame.from_dict(efs.get_metric_dict()).T
efs_results.sort_values(by='avg_score', ascending=True, inplace=True)
efs_results

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
1,"(1,)","[0.592, 0.576, 0.608, 0.5645161290322581, 0.60...",0.589071,"(Age,)",0.021425,0.016669,0.008335
11,"(1, 2)","[0.64, 0.568, 0.608, 0.5483870967741935, 0.612...",0.595458,"(Age, SibSp)",0.042299,0.03291,0.016455
12,"(1, 3)","[0.56, 0.592, 0.624, 0.6209677419354839, 0.612...",0.601974,"(Age, Parch)",0.03056,0.023777,0.011888
34,"(1, 3, 4)","[0.584, 0.6, 0.624, 0.5967741935483871, 0.6209...",0.605148,"(Age, Parch, Fare)",0.019488,0.015163,0.007581
13,"(1, 4)","[0.592, 0.616, 0.616, 0.5725806451612904, 0.63...",0.606735,"(Age, Fare)",0.028608,0.022258,0.011129
3,"(3,)","[0.568, 0.6, 0.616, 0.6048387096774194, 0.6451...",0.6068,"(Parch,)",0.032064,0.024947,0.012473
31,"(1, 2, 3)","[0.648, 0.576, 0.656, 0.5887096774193549, 0.62...",0.617935,"(Age, SibSp, Parch)",0.040542,0.031543,0.015771
2,"(2,)","[0.624, 0.6, 0.656, 0.5645161290322581, 0.7177...",0.632452,"(SibSp,)",0.066955,0.052094,0.026047
23,"(0, 1, 4)","[0.656, 0.632, 0.648, 0.6129032258064516, 0.65...",0.640426,"(Pclass, Age, Fare)",0.020663,0.016076,0.008038
42,"(0, 1, 2, 4)","[0.656, 0.648, 0.64, 0.6129032258064516, 0.653...",0.642026,"(Pclass, Age, SibSp, Fare)",0.019983,0.015548,0.007774


In [50]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0, criterion='gini')
rfc.fit(x_train, y_train)
for feature in zip(x_train.columns, rfc.feature_importances_):
    print(feature)

('Pclass', np.float64(0.08580548090109545))
('Age', np.float64(0.26433757901109173))
('SibSp', np.float64(0.05355992352286061))
('Parch', np.float64(0.03699908167792097))
('Fare', np.float64(0.28790097031129963))
('Sex_male', np.float64(0.2713969645757317))


In [52]:
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(estimator=rfc, threshold=0.2)
sfm = sfm.fit(x_train, y_train)
feature_names=x_train.columns
print('The most important features based on random forest classifier:')
for feature_list_index in sfm.get_support(indices=True):
    print('- ' + feature_names[feature_list_index])

The most important features based on random forest classifier:
- Age
- Fare
- Sex_male


In [54]:
from sklearn.metrics import accuracy_score
rfc_full = RandomForestClassifier(random_state=0, criterion='gini')
rfc_full.fit(x_train, y_train)
pred_full = rfc_full.predict(x_test)
print('The accuracy of classifier with full features: {:.2f}'.format(accuracy_score(y_test, pred_full)))

The accuracy of classifier with full features: 0.82


In [56]:
x_important_train = sfm.transform(x_train)
x_important_test = sfm.transform(x_test)

In [57]:
print(x_important_train.shape)
print(x_important_test.shape)

(623, 3)
(268, 3)


In [58]:
rfc_lim = RandomForestClassifier(random_state=0, criterion='gini')
rfc_lim.fit(x_important_train, y_train)
pred_lim = rfc_lim.predict(x_important_test)
print('The accuracy of classifier with limited features: {:.2f}'.format(accuracy_score(y_test, pred_lim)))

The accuracy of classifier with limited features: 0.80
